In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import p_power as pp
import torch as tc
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from keras import Input, optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import L1, L2, L1L2, OrthogonalRegularizer
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate
from sklearn import preprocessing, linear_model, tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier


Using device: cpu


2024-08-13 16:42:15.629894: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-13 16:42:15.646216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 16:42:15.666273: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 16:42:15.672496: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 16:42:15.688533: I tensorflow/core/platform/cpu_feature_guar

import numpy as np
from scipy.optimize import minimize

# Define matrix A
A = np.array([[1, 2], [3, 4], [5, 6]])

# Define the regularization parameter
lambda_ = 1.0

# Define the objective function
def objective(x):
    Ax = A @ x
    return (1 + lambda_) * np.linalg.norm(Ax)**2

# Initial guess for x
x0 = np.zeros(A.shape[1])

# Solve the optimization problem
result = minimize(objective, x0, method='L-BFGS-B')

# Optimal x
x_optimal = result.x
print("Optimal x:", x_optimal)

# Compute optimal b = Ax
b_optimal = A @ x_optimal
print("Optimal b:", b_optimal)


1. Read in Data

In [2]:
df = pd.read_csv('../data/l_pi.csv')


In [3]:
print(df.describe())


         velocity  weightpercentw  diametermm  thicknessmm    heightin  \
count  256.000000      256.000000  256.000000        256.0  256.000000   
mean     4.447500        0.375000    0.020000          0.0   43.750000   
std      1.421588        0.365578    0.007085          0.0   25.636983   
min      2.550000        0.100000    0.010000          0.0   13.000000   
25%      3.487500        0.137500    0.017500          0.0   25.000000   
50%      4.445000        0.200000    0.020000          0.0   40.500000   
75%      5.405000        0.437500    0.022500          0.0   59.250000   
max      6.350000        1.000000    0.030000          0.0   81.000000   

       craterdiameterfromouteredgesmm  craterdiameterfromouteredgesmmno  \
count                      256.000000                        256.000000   
mean                        53.556250                         64.258594   
std                         21.430322                         25.712146   
min                         13.50

2. Separate Column Paramters into separate int pd vects and scalars

In [4]:
out_full = df.iloc[:, 0]
vel = df.iloc[:, 2]
wp = df.iloc[:, 3]
d_mm = df.iloc[:, 4]
t_mm = df.iloc[:, 5]
sigma = df.iloc[:, 24]
nu = df.iloc[:, 25]
# pi1 = pd.to_numeric(df.iloc[:, 29], errors='coerce')
# pi2 = pd.to_numeric(df.iloc[:, 30], errors='coerce')
# pi3 = pd.to_numeric(df.iloc[:, 31], errors='coerce')
# pi4 = pd.to_numeric(df.iloc[:, 32], errors='coerce')
# pi5 = pd.to_numeric(df.iloc[:, 33], errors='coerce')
# pi6 = pd.to_numeric(df.iloc[:, 34], errors='coerce')
# pi7 = pd.to_numeric(df.iloc[:, 35], errors='coerce')
# pi8 = pd.to_numeric(df.iloc[:, 36], errors='coerce')


out_trunk = df.iloc[:, 44]

pi1 = df.iloc[:, 29]
pi2 = df.iloc[:, 30]
pi3 = df.iloc[:, 31]
pi4 = df.iloc[:, 32]
pi5 = df.iloc[:, 33]
# pi6 = df.iloc[:, 34]
# pi7 = df.iloc[:, 35]
# pi8 = df.iloc[:, 36]

print(out_trunk)

0                      Splash
1                      Splash
2                      Splash
3      Broken or Intact Sheet
4                      Splash
                ...          
251                      Lump
252                      Lump
253                      Lump
254                      Lump
255                      Lump
Name: newcat1, Length: 256, dtype: object


3. Concat pi groups together and the dimension paramters together

In [5]:
phi = tc.Tensor([[0, 0, 1, 1, 1], [-2, -2, -1, 1, 1], [2, 0, 0, 0, 0], [0, 2, 0, 0, 0], [0, 0, 2, 0, 0], [0, 0, 0, -2, -2], [0, 0, 0, 2, 0], [0, 0, 0, 0, 2]])
phi_T = phi.mT
phi_G = phi_T @ phi
print(phi_G)

x_targ = tc.Tensor([[-.5], [0], [1], [.5], [-.5]])
x_start = tc.Tensor(tc.randn(5, 1))
print(x_start)


phi_norm, vect = pp.p_power(phi, p=2, s_max=100, type=tc.float)
print(phi_norm)
print(vect)


for i in range(100):
    start_norm = tc.linalg.vector_norm(x_start, ord=2, keepdims=True)
    x_start = (phi_G @ x_start) / start_norm
    x_start = pp.dual(x_start, p=2, dim=0, type=tc.float)
print(x_start) #+ 2.28892574923842)
print(start_norm)

res = phi @ x_start
print(res)

res_norm = tc.linalg.vector_norm(res, ord=2, keepdims=True)

print(res_norm)

tensor([[ 8.,  4.,  2., -2., -2.],
        [ 4.,  8.,  2., -2., -2.],
        [ 2.,  2.,  6.,  0.,  0.],
        [-2., -2.,  0., 10.,  6.],
        [-2., -2.,  0.,  6., 10.]])
tensor([[-0.3978],
        [ 0.7664],
        [-0.1105],
        [ 0.1546],
        [ 0.5231]])
tensor([[[4.3193]]])
tensor([[[-0.3883],
         [-0.3882],
         [-0.1227],
         [ 0.5847],
         [ 0.5845]]])
tensor([[-0.3882],
        [-0.3882],
        [-0.1227],
        [ 0.5846],
        [ 0.5846]])
tensor([[1.]])
tensor([[ 1.0465],
        [ 2.8447],
        [-0.7764],
        [-0.7764],
        [-0.2454],
        [-2.3385],
        [ 1.1692],
        [ 1.1692]])
tensor([[4.3193]])


In [6]:
params = pd.concat([vel, wp, d_mm, t_mm, sigma, nu], axis=1)
# print(params)

tensor_params = tc.tensor(params.values, dtype=tc.float32)

phi_plus = tc.concat([phi, res], dim=1)
print(params.shape)
# pi_alg = (np.log(tensor_params)) @ 

(256, 6)


In [7]:
# pi_groups = pd.concat([pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8], axis=1)



pi_groups = pd.concat([pi1, pi2, pi3, pi4, pi5], axis=1)

print(pi_groups.dtypes)


print(pi_groups)

pi1    float64
pi2    float64
pi3    float64
pi4    float64
pi5    float64
dtype: object
              pi1           pi2          pi3        pi4       pi5
0    50991.962110  50541.334112  3105.712433  24.579584  0.061449
1     5766.579789   5615.703790  3105.712433  24.579584  0.553041
2     1479.863947   1403.925948  3105.712433  24.579584  2.212163
3      846.911845    789.708345  3105.712433  24.579584  3.932733
4    50991.962110  50541.334112  3105.712433  55.304064  0.061449
..            ...           ...          ...        ...       ...
251    185.134432    158.921609    61.240388   9.633741  0.385350
252  10373.685582  10170.982999    61.240388  15.052721  0.006021
253   1198.343416   1130.109222    61.240388  15.052721  0.054190
254    317.144403    282.527306    61.240388  15.052721  0.216759
255    185.134432    158.921609    61.240388  15.052721  0.385350

[256 rows x 5 columns]


4. Create the Model

In [8]:
def train_and_predict_using_model(model_name = "", model =None):
    model.fit(X_train, Y_train)
    Y_pred_train = model.predict(X_train)
    cm_train = confusion_matrix(Y_train, Y_pred_train)
    print(model_name)
    print("====================================")
    print("Training Confusion Matrix: ")
    print(cm_train)
    acc_train = accuracy_score(Y_train, Y_pred_train)
    
    print("Training Accuracy: %.2f%%" % (acc_train*100))
    print("====================================")
    
    Y_pred = model.predict(X_test)
    cm_test = confusion_matrix(Y_test, Y_pred)
    print("Testing Confusion Matrix: ")
    print(cm_test)
    acc_test = acc_train = accuracy_score(Y_test, Y_pred)
    
    print("Testing Accuracy: %.2f%%" % (acc_test*100))
    print("====================================")

In [9]:
def train_and_predict_using_pimodel(model_name = "", model =None):
    model.fit(scaled_X_train_pi, Y_train_pi)
    Y_pred_train_pi = model.predict(scaled_X_train_pi)
    cm_train_pi = confusion_matrix(Y_train_pi, Y_pred_train_pi)
    print(model_name)
    print("====================================")
    print("Training Confusion Matrix: ")
    print(cm_train_pi)
    acc_train = (np.trace(cm_train_pi)) / np.sum(np.sum(cm_train_pi))
    
    print("Training Accuracy: %.2f%%" % (acc_train*100))
    print("====================================")
    
    Y_pred_pi = model.predict(scaled_X_test_pi)
    cm_test_pi = confusion_matrix(Y_test_pi, Y_pred_pi)
    print("Testing Confusion Matrix: ")
    print(cm_test_pi)
    acc_test = acc_train = np.trace(cm_test_pi) / np.sum(np.sum(cm_test_pi))
    
    print("Testing Accuracy: %.2f%%" % (acc_test*100))
    print("====================================")

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(params, out_trunk, test_size=0.2, random_state=42)
X_train_pi, X_test_pi, Y_train_pi, Y_test_pi = train_test_split(pi_groups, out_trunk, test_size=0.2, random_state=0)

# scaler = preprocessing.StandardScaler().fit(X_train)
# scaled_X_train = scaler.transform(X_train)
# scaled_X_test = scaler.transform(X_test)

scaler_pi = preprocessing.StandardScaler().fit(X_train_pi)
scaled_X_train_pi = scaler_pi.transform(X_train_pi)
scaled_X_test_pi = scaler_pi.transform(X_test_pi)
encoder = LabelEncoder()
encoder.fit(out_trunk)
encoded_ytest_pi = encoder.transform(Y_test_pi)
encoded_ytrain_pi = encoder.transform(Y_train_pi)
encoded_y_pi = encoder.transform(out_trunk)
# One-hot encode the target variable
encoded_ytrain_pi_onehot = to_categorical(encoded_ytrain_pi)

encoded_ytest_pi_onehot = to_categorical(encoded_ytest_pi)
encoded_y_pi_onehot = to_categorical(encoded_y_pi)

In [11]:
scaler_dim = preprocessing.StandardScaler().fit(X_train)
scaled_X_train = scaler_dim.transform(X_train)
scaled_X_test = scaler_dim.transform(X_test)
encoder = LabelEncoder()
encoder.fit(out_trunk)
encoded_ytest = encoder.transform(Y_test)
encoded_ytrain = encoder.transform(Y_train)
encoded_y = encoder.transform(out_trunk)
# One-hot encode the target variable
encoded_ytrain_onehot = to_categorical(encoded_ytrain)

encoded_ytest_onehot = to_categorical(encoded_ytest)
encoded_y_onehot = to_categorical(encoded_y)

In [12]:

print(encoded_y_pi_onehot)



[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


5. Try some stuff


In [13]:
# Logistic Regression
linear_classifier = linear_model.LogisticRegression(random_state=123)
train_and_predict_using_model("Logistic Regression", linear_classifier)

Logistic Regression
Training Confusion Matrix: 
[[60  6  0 14]
 [11 13  0  0]
 [ 0  2 14  0]
 [23  0  0 61]]
Training Accuracy: 72.55%
Testing Confusion Matrix: 
[[10  4  0  6]
 [ 2  6  0  0]
 [ 0  1  2  0]
 [ 6  0  0 15]]
Testing Accuracy: 63.46%


/home/MoDMAN200o/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
linear_classifier_pi = linear_model.LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=.5, max_iter=1000)
# linear_classifier_pi = linear_model.LogisticRegression()
train_and_predict_using_pimodel("Pi Logistic Regression", linear_classifier_pi)
print("l1l2 Params: ", linear_classifier_pi.coef_)
linear_classifier_pi_l1 = linear_model.LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=1.0, max_iter=1000)
# linear_classifier_pi = linear_model.LogisticRegression()
train_and_predict_using_pimodel("Pi Logistic Regression", linear_classifier_pi_l1)
print("l1 Params: ", linear_classifier_pi_l1.coef_)

linear_classifier_pi_l2 = linear_model.LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.0, max_iter=1000)
# linear_classifier_pi = linear_model.LogisticRegression()
train_and_predict_using_pimodel("Pi Logistic Regression", linear_classifier_pi_l2)
print("l2 Params: ", linear_classifier_pi_l2.coef_)

Pi Logistic Regression
Training Confusion Matrix: 
[[71  0  0  7]
 [19  2  4  0]
 [ 2  4 10  0]
 [13  0  0 72]]
Training Accuracy: 75.98%
Testing Confusion Matrix: 
[[17  0  0  5]
 [ 6  1  0  0]
 [ 1  1  1  0]
 [ 4  0  0 16]]
Testing Accuracy: 67.31%
l1l2 Params:  [[ 0.19057381  0.21276515  0.88014905  0.15861865  1.17624153]
 [-0.11041279 -0.08238353 -1.39885644 -1.34333386  0.75485584]
 [-0.34370696 -0.3330054  -2.84871317 -1.58663487 -0.88725373]
 [ 0.26357445  0.20265228  3.3701357   2.77644867 -1.04323621]]
Pi Logistic Regression
Training Confusion Matrix: 
[[71  0  0  7]
 [17  4  4  0]
 [ 0  1 15  0]
 [12  0  0 73]]
Training Accuracy: 79.90%
Testing Confusion Matrix: 
[[17  0  0  5]
 [ 4  3  0  0]
 [ 0  1  2  0]
 [ 2  0  0 18]]
Testing Accuracy: 76.92%
l1 Params:  [[ 0.          0.35349088  2.53139948  0.47385502  1.10446956]
 [-0.15435676  0.         -0.63442938 -1.24685456  0.82230805]
 [-0.21793357 -0.11443419 -6.73198102 -1.90466554 -0.15517558]
 [ 0.40657534  0.          5.4

In [15]:
phi = tc.Tensor([[0, 0, 1, 1, 1], [-2, -2, -1, 1, 1], [2, 0, 0, 0, 0], [0, 2, 0, 0, 0], [0, 0, 2, 0, 0], [0, 0, 0, -2, -2], [0, 0, 0, 2, 0], [0, 0, 0, 0, 2]])
x_targ = tc.Tensor([[-.5], [0], [1], [.5], [-.5]])
phi_norm, vect = pp.p_power(phi, p=2, s_max=500, type=tc.float)
reg_phi = phi / phi_norm
l2_coef = tc.Tensor(linear_classifier_pi_l2.coef_)
l2_martix = (l2_coef @ reg_phi.mT) @ (phi @ x_targ)
print(l2_martix)
print((pp.p_power(l2_martix[0, :, :], p=2, s_max=500, type=tc.float)[0]) /  1)#2.28892574923842)
# print(vect)

tensor([[[ 0.4530],
         [-2.5806],
         [-3.0908],
         [ 5.2185]]])
tensor([[[6.6069]]])


Decision Tree



In [16]:
# Decision Tree
decision_tree_clf = tree.DecisionTreeClassifier()
train_and_predict_using_model('Decision Tree', decision_tree_clf)

Decision Tree
Training Confusion Matrix: 
[[73  2  0  5]
 [ 9 15  0  0]
 [ 0  0 16  0]
 [24  0  0 60]]
Training Accuracy: 80.39%
Testing Confusion Matrix: 
[[12  3  0  5]
 [ 3  4  1  0]
 [ 0  0  3  0]
 [ 7  0  0 14]]
Testing Accuracy: 63.46%


In [17]:
# Decision Tree and Random Forests
decision_tree_clf_pi = tree.DecisionTreeClassifier()
train_and_predict_using_pimodel('Decision Tree', decision_tree_clf_pi)

Decision Tree
Training Confusion Matrix: 
[[78  0  0  0]
 [ 0 25  0  0]
 [ 0  0 16  0]
 [ 0  0  0 85]]
Training Accuracy: 100.00%
Testing Confusion Matrix: 
[[14  3  0  5]
 [ 3  3  1  0]
 [ 0  1  2  0]
 [ 2  0  0 18]]
Testing Accuracy: 71.15%


In [18]:
model4 = GradientBoostingClassifier(n_estimators=80, learning_rate=.7, max_depth=9, random_state=357, loss='log_loss', criterion='friedman_mse', min_samples_split=2, min_samples_leaf=2, max_features=4, max_leaf_nodes=None, min_impurity_decrease=.000001, init=None, subsample=1.0)
train_and_predict_using_pimodel('Gradient Boosting', model4)

# 93.515% accuracy

# model4 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.20, max_depth=6, random_state=0, loss='log_loss', criterion='squared_error', min_samples_split=3, min_samples_leaf=2, max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0, init=None, subsample=1.0)
# train_and_predict_using_pimodel('Gradient Boosting', model4)

# 93.75% accuracy

# model4 = GradientBoostingClassifier(n_estimators=250, learning_rate=1.255, max_depth=9, random_state=0, loss='log_loss', criterion='friedman_mse', min_samples_split=4, min_samples_leaf=2, max_features=4, max_leaf_nodes=None, min_impurity_decrease=0, init=None, subsample=1.0)
# train_and_predict_using_pimodel('Gradient Boosting', model4)

# model4 = GradientBoostingClassifier(n_estimators=300, learning_rate=1.3, max_depth=12, random_state=357, loss='log_loss', criterion='friedman_mse', min_samples_split=4, min_samples_leaf=2, max_features=4, max_leaf_nodes=None, min_impurity_decrease=0, init=None, subsample=1.0)
# train_and_predict_using_pimodel('Gradient Boosting', model4)

Gradient Boosting
Training Confusion Matrix: 
[[78  0  0  0]
 [ 0 25  0  0]
 [ 0  0 16  0]
 [ 0  0  0 85]]
Training Accuracy: 100.00%
Testing Confusion Matrix: 
[[17  1  0  4]
 [ 3  4  0  0]
 [ 0  0  3  0]
 [ 3  0  0 17]]
Testing Accuracy: 78.85%


Random Forest

In [19]:
forest = RandomForestClassifier(n_estimators=100, random_state=123, max_depth=8, max_features=6)
train_and_predict_using_model('Random Forest', forest)

Random Forest
Training Confusion Matrix: 
[[69  3  0  8]
 [ 8 16  0  0]
 [ 0  0 16  0]
 [21  0  0 63]]
Training Accuracy: 80.39%
Testing Confusion Matrix: 
[[ 9  4  0  7]
 [ 3  4  1  0]
 [ 0  0  3  0]
 [ 6  1  0 14]]
Testing Accuracy: 57.69%


In [20]:
forest = RandomForestClassifier(n_estimators=150, random_state=123, max_depth=6, max_features=3, criterion='entropy', min_weight_fraction_leaf=0.0, max_leaf_nodes=None )

train_and_predict_using_pimodel('Random Forest', forest)

Random Forest
Training Confusion Matrix: 
[[76  0  0  2]
 [ 1 24  0  0]
 [ 0  0 16  0]
 [ 1  0  0 84]]
Training Accuracy: 98.04%
Testing Confusion Matrix: 
[[18  1  0  3]
 [ 1  6  0  0]
 [ 0  1  2  0]
 [ 1  0  0 19]]
Testing Accuracy: 86.54%


Neural Net


In [21]:
pi1_wghts = tf.exp(tf.constant([1.0, -1.0, 2.0, 0.0, 0.0, 0.0])) / tf.linalg.norm(tf.exp(tf.constant([1.0, -1.0, 2.0, 0.0, 0.0, 0.0])))
pi2_wghts = tf.exp(tf.constant([1.0, 1.0, 0.0, -2.0, -2.0, 0.0])) / tf.linalg.norm(tf.exp(tf.constant([1.0, 1.0, 0.0, -2.0, -2.0, 0.0])))
pi3_wghts =tf.exp(tf.constant([1.0, 1.0, 0.0, -2.0, 0.0, 2.0])) / tf.linalg.norm(tf.exp(tf.constant([1.0, 1.0, 0.0, -2.0, 0.0, 2.0])))

pi_weights = tf.concat([[pi1_wghts], [pi2_wghts], [pi3_wghts]], 0)

print(pi_weights.shape)

(3, 6)


In [45]:
def train_and_predict_dimnet(opt=None, model=None, verbose=1):
    
    if opt is None:
        model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    else:
    
        model.compile(loss='categorical_focal_crossentropy', optimizer=opt, metrics=['categorical_crossentropy'])
    
    model.fit(scaled_X_train, encoded_ytrain_onehot, epochs=250, verbose=0)
    
    
    Y_pred_train = model.predict(scaled_X_train)

    # Evaluate the model
    scores = model.evaluate(scaled_X_train, encoded_ytrain_onehot)

    print("Neural Network Trainset: \n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

    Y_pred = model.predict(scaled_X_test)

    # Convert predictions to class labels
    Y_pred_labels = np.argmax(Y_pred, axis=1)
    Y_test_labels = np.argmax(encoded_ytest_onehot, axis=1)
    # import pdb; pdb.set_trace()
    Y_pred_train_labels = np.argmax(Y_pred_train, axis=1)
    Y_train_labels = np.argmax(encoded_ytrain_onehot, axis=1)
    if verbose:    
        print("Summary of Neural Network:")
        print(model.summary())
        
        print("====================================")
    
        cm_train = confusion_matrix(Y_train_labels, Y_pred_train_labels)
        # print("====================================")
        print("Training Confusion Matrix: ")
        print(cm_train)
        acc_train = accuracy_score(Y_train_labels, Y_pred_train_labels)
        print("Training Accuracy: %.2f%%" % (acc_train * 100))
        print("====================================")
        
        print("Training Classification Report")
        print(classification_report(Y_train_labels, Y_pred_train_labels))
        
        print("====================================")
        
        
        
        cm_test = confusion_matrix(Y_test_labels, Y_pred_labels)
        print("Testing Confusion Matrix: ")
        print(cm_test)
        acc_test = accuracy_score(Y_test_labels, Y_pred_labels)

        print("Testing Accuracy: %.2f%%" % (acc_test * 100))
        print("====================================")
        
        print("Testing Classification Report")
        print(classification_report(Y_test_labels, Y_pred_labels))
        
        print("====================================")
        
        
        
        return Y_test_labels, Y_pred_labels
    
    acc_train = accuracy_score(Y_train_labels, Y_pred_train_labels)
    acc_test = accuracy_score(Y_test_labels, Y_pred_labels)
    
    return acc_train, acc_test
    # print(Y_pred_pi)

In [46]:
def train_and_predict_pinets(opt=None, model=None, verbose=1):
    
    if opt is None:
        model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    else:
    
        model.compile(loss='categorical_focal_crossentropy', optimizer=opt, metrics=['categorical_crossentropy'])
    
    model.fit(scaled_X_train_pi, encoded_ytrain_pi_onehot, epochs=250,  verbose=0)
    
    
    Y_pred_train_pi = model.predict(scaled_X_train_pi)

    # Evaluate the model
    scores = model.evaluate(scaled_X_train_pi, encoded_ytrain_pi_onehot)

    print("Neural Network Trainset: \n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

    Y_pred_pi = model.predict(scaled_X_test_pi)

    # Convert predictions to class labels
    Y_pred_pi_labels = np.argmax(Y_pred_pi, axis=1)
    Y_test_pi_labels = np.argmax(encoded_ytest_pi_onehot, axis=1)
    # import pdb; pdb.set_trace()
    Y_pred_train_pi_labels = np.argmax(Y_pred_train_pi, axis=1)
    Y_train_pi_labels = np.argmax(encoded_ytrain_pi_onehot, axis=1)
    if verbose:
        print("Summary of Neural Network:")
        print(model.summary())
        
        print("====================================")
    
        cm_train = confusion_matrix(Y_train_pi_labels, Y_pred_train_pi_labels)
        # print("====================================")
        print("Training Confusion Matrix: ")
        print(cm_train)
        acc_train = accuracy_score(Y_train_pi_labels, Y_pred_train_pi_labels)
        print("Training Accuracy: %.2f%%" % (acc_train * 100))
        print("====================================")
        
        print("Training Classification Report")
        print(classification_report(Y_train_pi_labels, Y_pred_train_pi_labels))
        
        print("====================================")
        
        
        
        cm_test = confusion_matrix(Y_test_pi_labels, Y_pred_pi_labels)
        print("Testing Confusion Matrix: ")
        print(cm_test)
        acc_test = accuracy_score(Y_test_pi_labels, Y_pred_pi_labels)

        print("Testing Accuracy: %.2f%%" % (acc_test * 100))
        print("====================================")
        
        print("Testing Classification Report")
        print(classification_report(Y_test_pi_labels, Y_pred_pi_labels))
        
        print("====================================")
        
        
    
        return Y_test_pi_labels, Y_pred_pi_labels
    
    acc_train = accuracy_score(Y_train_pi_labels, Y_pred_train_pi_labels)
    acc_test = accuracy_score(Y_test_pi_labels, Y_pred_pi_labels)
    
    return acc_train, acc_test
    

    # print(Y_pred_pi)

In [47]:
# Neural Network
def create_pinet(comp = False, dim_less = False):

    pi_net = Sequential()
   
    # 92.31% accuracy
    # pi_net.add(Input(shape=(pi_groups.shape[1],)))
    # pi_net.add(Dense(8, kernel_regularizer=OrthogonalRegularizer(factor=.01, mode="rows"),  name='orthog_Row'))
    # pi_net.add(Dense(5, kernel_regularizer=OrthogonalRegularizer(pi_groupsfactor=.01, mode="columns"),  name='orthog_Col'))
    if dim_less is True:
        pi_net.add(Input(shape=(pi_groups.shape[1],)))
        pi_net.add(Dense(5, name='pi')) #, activation='silu'))
    else:
        pi_net.add(Input(shape=(params.shape[1],), name='input'))
        
    pi_net.add(Dense(12, activation='silu', name='silu'))
    pi_net.add(Dense(24, activation='tanh', name = 'tanh'))
    pi_net.add(Dense(48, activation='leaky_relu'))
    pi_net.add(Dropout(0.2))
    pi_net.add(Dense(4, activation='softmax'))
    if comp is True:
        pi_net.compile(loss='categorical_focal_crossentropy', opt=optimizers.Adagrad(), metrics=['categorical_crossentropy'])
    
    return pi_net




In [34]:
dim = create_pinet(comp=False, dim_less=False)
print("Adam Optimizer")
train_and_predict_dimnet(model=dim, opt=optimizers.Adam())
print("AdamW Optimizer")
train_and_predict_dimnet(model=dim, opt=optimizers.AdamW())
print("Adagrad Optimizer")
train_and_predict_dimnet(model=dim, opt=optimizers.Adagrad())
print("Adadelta Optimizer")
train_and_predict_dimnet(model=dim, opt=optimizers.Adadelta())
print("RMSprop Optimizer")
train_and_predict_dimnet(model=dim, opt=optimizers.RMSprop())
print("SGD Optimizer")
train_and_predict_dimnet(model=dim, opt=optimizers.SGD())
print("Nadam Optimizer")
train_and_predict_dimnet(model=dim, opt=optimizers.Nadam())
# print("Ftrl Optimizer")
# train_and_predict_pinets(model=pim, opt=optimizers.Ftrl())
print("Adafactor Optimizer")
train_and_predict_dimnet(model=dim, opt=optimizers.Adafactor())
print("Adamax Optimizer")
train_and_predict_dimnet(model=dim, opt=optimizers.Adamax())
# print("Lion Optimizer")
# train_and_predict_pinets(model=pim, opt=optimizers.Lion())
print("Loss Scale Optimizer")
train_and_predict_dimnet(model=dim, opt=optimizers.LossScaleOptimizer(inner_optimizer=optimizers.Adam()))
# print("Lookahead Optimizer")
# train_and_predict_pinets(model=pim, opt=optimizers.Lookahead())





Adam Optimizer


KeyboardInterrupt: 

In [35]:
pim = create_pinet(comp=False, dim_less=True)
print("Adam Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.Adam())
print("AdamW Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.AdamW())
print("Adagrad Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.Adagrad())
print("Adadelta Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.Adadelta())
print("RMSprop Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.RMSprop())
print("SGD Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.SGD())
print("Nadam Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.Nadam())
# print("Ftrl Optimizer")
# train_and_predict_pinets(model=pim, opt=optimizers.Ftrl())
print("Adafactor Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.Adafactor())
print("Adamax Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.Adamax())
# print("Lion Optimizer")
# train_and_predict_pinets(model=pim, opt=optimizers.Lion())
print("Loss Scale Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.LossScaleOptimizer(inner_optimizer=optimizers.Adam()))
# print("Lookahead Optimizer")
# train_and_predict_pinets(model=pim, opt=optimizers.Lookahead())





Adam Optimizer
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_crossentropy: 0.3741 - loss: 0.0203  
Neural Network Trainset: 
compile_metrics: 35.51%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Summary of Neural Network:


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ pi (Dense)                      │ (None, 5)              │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ silu (Dense)                    │ (None, 12)             │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tanh (Dense)                    │ (None, 24)             │           312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 48)             │         1,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 4)              │           196 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,432 (21.22 KB)

 Trainable params: 1,810 (7.07 KB)

...

 Trainable params: 1,810 (7.07 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,626 (14.18 KB)

None
Training Confusion Matrix: 
[[76  1  0  1]
 [ 0 25  0  0]
 [ 0  0 16  0]
 [ 0  0  0 85]]
Training Accuracy: 99.02%
Training Classification Report
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        78
           1       0.96      1.00      0.98        25
           2       1.00      1.00      1.00        16
           3       0.99      1.00      0.99        85

    accuracy                           0.99       204
   macro avg       0.99      0.99      0.99       204
weighted avg       0.99      0.99      0.99       204

Testing Confusion Matrix: 
[[18  1  0  3]
 [ 0  7  0  0]
 [ 0  0  3  0]
 [ 2  0  0 18]]
Testing Accuracy: 88.46%
Testing Classification Report
              precision    recall  f1-score   support

           0       0.90      0.82      0.86        22
           1       0.88      1.00      0.93         7
           2       1.00      1.00      1.00         3
           3       0.86      0.90      0.88        20


(array([3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 3, 1, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3,
        2, 3, 0, 2, 3, 0, 0, 3, 3, 0, 0, 0, 3, 0, 1, 3, 3, 1, 0, 3, 1, 2,
        3, 0, 0, 3, 3, 1, 3, 1]),
 array([0, 0, 0, 0, 0, 3, 1, 3, 0, 0, 3, 1, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3,
        2, 3, 0, 2, 3, 0, 1, 3, 3, 0, 3, 0, 3, 0, 1, 3, 3, 1, 0, 3, 1, 2,
        3, 3, 0, 3, 3, 1, 0, 1]))

In [53]:
p = 10

# print("RMSprop Optimizer")
# print("====================================")
# print("Dimensionless:")
pim = create_pinet(comp=False, dim_less=True)
# train_and_predict_pinets(model=pim, opt=optimizers.RMSprop())
# print("====================================")
# print("Dimensional:")
dim = create_pinet(comp=False, dim_less=False)
# train_and_predict_dimnet(model=dim, opt=optimizers.RMSprop())
model_order = ['RMSprop', 'Adam', 'AdamW'] #, 'Adagrad', 'Adadelta', 'SGD', 'Nadam', 'Adafactor', 'Adamax', 'LossScaleOptimizer(Adam)']
num_models = len(model_order)
pi_scores = np.zeros((p, num_models, 2))
dim_scores = np.zeros((p, num_models, 2))
for i in range(p):
    pi_scores[i,:, :] = [train_and_predict_pinets(model=pim, opt=optimizers.RMSprop(), verbose=0), train_and_predict_pinets(model=pim, opt=optimizers.Adam(), verbose=0),
                         train_and_predict_pinets(model=pim, opt=optimizers.AdamW(), verbose=0)] #, train_and_predict_pinets(model=pim, opt=optimizers.Adagrad(), verbose=0),
                        #  train_and_predict_pinets(model=pim, opt=optimizers.Adadelta(), verbose=0), train_and_predict_pinets(model=pim, opt=optimizers.SGD(), verbose=0),
                        #  train_and_predict_pinets(model=pim, opt=optimizers.Nadam(), verbose=0), train_and_predict_pinets(model=pim, opt=optimizers.Adafactor(), verbose=0),
                        #  train_and_predict_pinets(model=pim, opt=optimizers.Adamax(), verbose=0), train_and_predict_pinets(model=pim, opt=optimizers.LossScaleOptimizer(inner_optimizer=optimizers.Adam()), verbose=0)]

    dim_scores[i, :, :] = [train_and_predict_dimnet(model=dim, opt=optimizers.RMSprop(), verbose=0), train_and_predict_dimnet(model=dim, opt=optimizers.Adam(), verbose=0),
                           train_and_predict_dimnet(model=dim, opt=optimizers.AdamW(), verbose=0)] #, train_and_predict_dimnet(model=dim, opt=optimizers.Adagrad(), verbose=0),
                        #    train_and_predict_dimnet(model=dim, opt=optimizers.Adadelta(), verbose=0), train_and_predict_dimnet(model=dim, opt=optimizers.SGD(), verbose=0),
                        #    train_and_predict_dimnet(model=dim, opt=optimizers.Nadam(), verbose=0), train_and_predict_dimnet(model=dim, opt=optimizers.Adafactor(), verbose=0),
                        #    train_and_predict_dimnet(model=dim, opt=optimizers.Adamax(), verbose=0), train_and_predict_dimnet(model=dim, opt=optimizers.LossScaleOptimizer(inner_optimizer=optimizers.Adam()), verbose=0)]
# pi_train_scores_df = pd.DataFrame(pi_scores[:, :, 0], columns=model_order)
# pi_test_scores_df = pd.DataFrame(pi_scores[:, :, 1], columns=model_order)
# dim_train_scores_df = pd.DataFrame(dim_scores[:, :, 0], columns=model_order)
# dim_test_scores_df = pd.DataFrame(dim_scores[:, :, 1], columns=model_order)
# pi_scores_df = pd.concat([pi_train_scores_df, pi_test_scores_df], axis=1)
# dim_scores_df = pd.concat([dim_train_scores_df, dim_test_scores_df], axis=1)

# Flatten the array to 2D
flattened_pi_data = pi_scores.reshape(p * num_models, 2)
flattened_dim_data = dim_scores.reshape(p * num_models, 2)

# Create a DataFrame
pi_df = pd.DataFrame(flattened_pi_data, columns=['Train', 'Test'])
dim_df = pd.DataFrame(flattened_dim_data, columns=['Train', 'Test'])
# Add columns for Trial and Model indices
pi_df['Trial'] = np.repeat(np.arange(p), num_models)
pi_df['Model'] = np.tile(model_order, p)

dim_df['Trial'] = np.repeat(np.arange(p), num_models)
dim_df['Model'] = np.tile(model_order, p)
# Reorder columns to have Trial and Model first
pi_df = pi_df[['Trial', 'Model', 'Train', 'Test']]
dim_df = dim_df[['Trial', 'Model', 'Train', 'Test']]

# Write the DataFrame to a CSV file
pi_df.to_csv('../out/out_pi.csv', index=False)
dim_df.to_csv('../out/out_dim.csv', index=False)




# pi_scores_df.to_csv('../out/pi_scores.csv', index=False)
# dim_scores_df.to_csv('../out/dim_scores.csv', index=False)



7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_crossentropy: 0.3596 - loss: 0.0207  
Neural Network Trainset: 
compile_metrics: 34.51%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_crossentropy: 0.2707 - loss: 0.0123  
Neural Network Trainset: 
compile_metrics: 25.99%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - categorical_crossentropy: 0.2243 - loss: 0.0092  
Neural Network Trainset: 
compile_metrics: 21.48%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_crossentropy: 0.4045 - loss: 0.0223  
Neural Network Trainset: 
compile_metrics: 43.46%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - categorical_crossentropy: 0.3855 - loss: 0.0214
Neural Network 

In [38]:
pi_layer = pim.get_layer('pi').get_weights()
# print(silu_layer[0])
pi_layer_tensor = tc.tensor(pi_layer[0])
print(pi_layer_tensor)
res, vect = pp.p_power(pi_layer_tensor, p=2, type=tc.float)
print(res)
print(vect)

tensor([[ 0.0957, -0.5894, -0.6761, -0.7326,  0.0698],
        [ 0.0685,  0.4181, -0.8529, -0.3151,  0.5515],
        [-0.6672, -0.4190,  0.1255,  0.6854,  0.0602],
        [-0.8934,  0.8771, -0.1478,  0.1477, -0.2753],
        [ 0.0095,  0.1448,  0.7403,  0.3503,  0.6252]])
tensor([[[1.6880]]])
tensor([[[-0.3748],
         [ 0.2191],
         [ 0.6467],
         [ 0.6252],
         [-0.0487]]])


In [152]:
d_pi = pi_layer_tensor.T @ pi_layer_tensor

print(d_pi) 
res, _ = pp.p_power(d_pi, p=2, type=tc.float)
print(res)

tensor([[ 1.5017, -0.2896, -0.5424,  1.5757,  0.6624],
        [-0.2896,  2.0219, -0.0666, -1.3330,  0.2950],
        [-0.5424, -0.0666,  2.8704,  0.2068, -0.7483],
        [ 1.5757, -1.3330,  0.2068,  3.3156,  0.5719],
        [ 0.6624,  0.2950, -0.7483,  0.5719,  0.6852]])
tensor([[[4.9388]]])


In [153]:
silu_layer = pim.get_layer('silu').get_weights()
# print(silu_layer[0])
silu_layer_tensor = tc.Tensor(silu_layer[0])
print(silu_layer_tensor.shape)
res, _ = pp.p_power(silu_layer_tensor, p=2, type=tc.float)
print(res)

torch.Size([5, 12])
tensor([[[2.8333]]])


In [111]:
print("Loss Scale Optimizer")
train_and_predict_pinets(model=pim, opt=optimizers.LossScaleOptimizer(inner_optimizer=optimizers.RMSprop()))

Loss Scale Optimizer
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_crossentropy: 0.0047 - loss: 1.7845e-06  
Neural Network Trainset: 
compile_metrics: 0.42%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Summary of Neural Network:


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_75 (Dense)                │ (None, 5)              │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_76 (Dense)                │ (None, 12)             │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_77 (Dense)                │ (None, 24)             │           312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_78 (Dense)                │ (None, 48)             │         1,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ (None, 4)              │           196 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,626 (14.18 KB)

 Trainable params: 1,810 (7.07 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,816 (7.11 KB)

None
Training Confusion Matrix: 
[[78  0  0  0]
 [ 0 25  0  0]
 [ 0  0 16  0]
 [ 0  0  0 85]]
Training Accuracy: 100.00%
Training Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        78
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        16
           3       1.00      1.00      1.00        85

    accuracy                           1.00       204
   macro avg       1.00      1.00      1.00       204
weighted avg       1.00      1.00      1.00       204

Testing Confusion Matrix: 
[[18  0  1  3]
 [ 1  6  0  0]
 [ 0  0  3  0]
 [ 2  0  0 18]]
Testing Accuracy: 86.54%
Testing Classification Report
              precision    recall  f1-score   support

           0       0.86      0.82      0.84        22
           1       1.00      0.86      0.92         7
           2       0.75      1.00      0.86         3
           3       0.86      0.90      0.88        20

(array([3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 3, 1, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3,
        2, 3, 0, 2, 3, 0, 0, 3, 3, 0, 0, 0, 3, 0, 1, 3, 3, 1, 0, 3, 1, 2,
        3, 0, 0, 3, 3, 1, 3, 1]),
 array([0, 0, 0, 0, 0, 3, 1, 3, 0, 0, 3, 1, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3,
        2, 3, 3, 2, 3, 0, 0, 3, 3, 0, 0, 0, 3, 0, 1, 3, 3, 0, 0, 3, 1, 2,
        3, 3, 2, 3, 3, 1, 0, 1]))

In [110]:
ortho_layer_wShape = (pim.get_layer('dense_54').get_weights()[0]).shape

print(ortho_layer_wShape)

ValueError: No such layer: dense_54. Existing layers are: ['dense_75', 'dense_76', 'dense_77', 'dense_78', 'dropout_15', 'dense_79'].

In [78]:

silu_layer = pim.get_layer('silu').get_weights()
# print(silu_layer[0])
silu_layer_tensor = tc.complex(tc.tensor(silu_layer[0]).T, tc.zeros_like(tc.tensor(silu_layer[0]).T))
print(silu_layer_tensor.shape)
res, _ = pp.p_power(silu_layer_tensor, p=2)
print(res)


gelu_layer = pim.get_layer('gelu').get_weights()
# print(silu_layer[0])
gelu_layer_tensor = tc.complex(tc.tensor(gelu_layer[0]).T, tc.zeros_like(tc.tensor(gelu_layer[0]).T))
print(gelu_layer_tensor.shape)
res, _ = pp.p_power(gelu_layer_tensor, p=2)
print(res)

v = tc.linalg.vector_norm(gelu_layer_tensor[:,31], ord=2)
print('Gelu Col: ',v)

softmax_layer = pim.get_layer('softmax').get_weights()
# print(silu_layer[0])
softmax_layer_tensor = tc.complex(tc.tensor(softmax_layer[0]).T, tc.zeros_like(tc.tensor(softmax_layer[0]).T))
print(softmax_layer_tensor.shape)
res, _ = pp.p_power(softmax_layer_tensor, p=2)
print(res)

ValueError: No such layer: silu. Existing layers are: ['dense_54', 'dense_55', 'dense_56'].

In [ ]:
# 90.625% accuracy
""" pi_net.add(Input(shape=(6,))) 
pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(256, activation='tanh'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(256, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(4, activation='softmax')) """

In [ ]:
# 92.19% accuracy
""" pi_net.add(Input(shape=(6,))) 
pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(256, activation='gelu'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(256, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(16))
pi_net.add(Dense(4, activation='softmax')) """

In [733]:
"""  92.31% accuracy
    pi_net.add(Input(shape=(6,)))
    pi_net.add(Dense(32, activation='relu'))
    pi_net.add(Dropout(0.3))
    pi_net.add(Dense(64, activation='relu'))
    pi_net.add(Dropout(0.3))
    pi_net.add(Dense(32, activation='leaky_relu'))
    pi_net.add(Dense(64, activation='leaky_relu'))
    pi_net.add(Dropout(0.3))
    pi_net.add(Dense(128, activation='tanh'))
    pi_net.add(Dropout(0.5))
    pi_net.add(Dense(64, activation='relu'))
    pi_net.add(Dropout(0.5))
    pi_net.add(Dense(32, activation='gelu'))
    pi_net.add(Dense(4, activation='softmax')) """

"  92.31% accuracy\n    pi_net.add(Input(shape=(6,)))\n    pi_net.add(Dense(32, activation='relu'))\n    pi_net.add(Dropout(0.3))\n    pi_net.add(Dense(64, activation='relu'))\n    pi_net.add(Dropout(0.3))\n    pi_net.add(Dense(32, activation='leaky_relu'))\n    pi_net.add(Dense(64, activation='leaky_relu'))\n    pi_net.add(Dropout(0.3))\n    pi_net.add(Dense(128, activation='tanh'))\n    pi_net.add(Dropout(0.5))\n    pi_net.add(Dense(64, activation='relu'))\n    pi_net.add(Dropout(0.5))\n    pi_net.add(Dense(32, activation='gelu'))\n    pi_net.add(Dense(4, activation='softmax')) "

In [ ]:
# 93.75% accuracy
""" pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(64, activation='relu'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(32, activation='gelu'))
pi_net.add(Dense(4, activation='softmax')) """

In [ ]:
""" # 94.23% accuracy
    pi_net.add(Input(shape=(6,)))
    pi_net.add(Dense(32, activation='silu'))
    pi_net.add(Dense(64, activation='silu'))
    pi_net.add(Dense(128, activation='tanh'))
    pi_net.add(Dropout(0.25))
    pi_net.add(Dense(256, activation='leaky_relu'))
    pi_net.add(Dense(512, activation='silu'))
    pi_net.add(Dropout(0.5))
    pi_net.add(Dense(64, activation='leaky_relu'))
    pi_net.add(Dropout(0.25))
    pi_net.add(Dense(32, activation='gelu'))
    pi_net.add(Dense(4, activation='softmax')) """

In [ ]:
#  # 94.23% accuracy
#     pi_net.add(Input(shape=(8,)))
#     # pi_net.add(Dense(32, activation='silu'))
#     pi_net.add(Dense(64, activation='silu'))
#     pi_net.add(Dropout(0.35))
#     pi_net.add(Dense(128, activation='tanh'))
#     pi_net.add(Dropout(0.5))
#     pi_net.add(Dense(64, activation='elu'))
    
#     # pi_net.add(Dense(32, activation='gelu'))
#     pi_net.add(Dense(4, activation='softmax'))

In [ ]:
# # 94.23% accuracy more consistent
#     pi_net = Sequential()
   
#     # 94.23% accuracy
#     pi_net.add(Input(shape=(6,)))
#     # pi_net.add(Dense(32, activation='silu'))
#     pi_net.add(Dense(64, activation='silu'))
#     pi_net.add(Dropout(0.35))
#     pi_net.add(Dense(128, activation='tanh'))
#     pi_net.add(Dropout(0.5))
#     pi_net.add(Dense(64, activation='elu'))
    
#     # pi_net.add(Dense(32, activation='gelu'))
#     pi_net.add(Dense(4, activation='softmax'))

In [ ]:
    # # 94.23% accuracy
    # pi_net.add(Input(shape=(6,)))
    # pi_net.add(Dense(32, activation='silu'))
    # pi_net.add(Dense(64, activation='tanh'))
    # # pi_net.add(Dense(128, activation='leaky_relu'))
    # pi_net.add(Dropout(0.5))
    # pi_net.add(Dense(64, activation='elu'))
    # pi_net.add(Dropout(0.5))
    # pi_net.add(Dense(32, activation='gelu'))
    # pi_net.add(Dense(4, activation='softmax'))
    # if comp is True:

In [150]:
# 95.83% accuracy
""" pi_net.add(Input(shape=(6,))) 
pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(256, activation='tanh'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(32, activation='gelu'))
pi_net.add(Dense(16, activation='leaky_relu'))
pi_net.add(Dense(4, activation='softmax')) """

In [ ]:
# 96.15% accuracy 2 wrong
# pi_net.add(Dense(32, activation='silu'))
# pi_net.add(Dense(64, activation='tanh'))
# pi_net.add(Dense(128, activation='leaky_relu'))
# pi_net.add(Dropout(0.5))
# pi_net.add(Dense(64, activation='relu'))
# pi_net.add(Dropout(0.5))
# pi_net.add(Dense(32, activation='gelu'))
# pi_net.add(Dense(4, activation='softmax'))


Keras Classifier

In [101]:
# Use KerasClassifier for scikit-learn compatibility
net = create_pinet(False)
Y_test_pi_labels, Y_pred_pi_labels = train_and_predict_pinets(model=net, opt=optimizers.Nadam())
model = KerasClassifier(model=net, epochs=500, batch_size=-1, verbose=1, metrics=['categorical_accuracy'])

# Perform cross-validation
kfold = KFold(n_splits=4, shuffle=True)
results = cross_validate(model, X=pi_groups, y=encoded_y_pi_onehot, cv=kfold, scoring='accuracy')
# recall_score(y_true=Y_test_pi_labels, y_pred=Y_pred_pi_labels, average='weighted')
print(f'Cross-Validation Accuracy: {results.mean():.2f} (+/- {results.std():.2f})')

# train_and_predict_pinets(model=create_pinet(True), opt='adam')

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.7714 - loss: 0.5576  
Neural Network Trainset: 
compile_metrics: 75.95%
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Testing Confusion Matrix: 
[[32  0  0  9]
 [12  0  0  0]
 [ 5  0  0  0]
 [ 2  0  0 38]]
Testing Accuracy: 71.43%
Classification Report
              precision    recall  f1-score   support

           0       0.63      0.78      0.70        41
           1       0.00      0.00      0.00        12
           2       0.00      0.00      0.00         5
           3       0.81      0.95      0.87        40

    accuracy                           0.71        98
   macro avg       0.36      0.43      0.39        98
weighted avg       0.59      0.71      0.65        98

Summary of Neural Network:


/home/MoDMAN200o/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/MoDMAN200o/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/MoDMAN200o/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ orthog_Row (Dense)              │ (None, 8)              │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ orthog_Col (Dense)              │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 4)              │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 4)              │            20 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,139 (266.17 KB)

 Trainable params: 22,712 (88.72 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 45,427 (177.45 KB)

None
Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - categorical_accuracy: 0.4219 - loss: 2.3000
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - categorical_accuracy: 0.4010 - loss: 2.2293
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - categorical_accuracy: 0.3438 - loss: 2.2966
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - categorical_accuracy: 0.3438 - loss: 2.3529
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - categorical_accuracy: 0.3490 - loss: 2.3104
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - categorical_accuracy: 0.3594 - loss: 2.3138
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - categorical_accuracy: 0.3542 - loss: 2.2615
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - categorical_accuracy: 0.3698 - loss: 2.1922
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - categorical_accuracy: 0.3438 - loss: 2.1368
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - categorical_accuracy: 0.3750 - loss: 2.0404
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━

AttributeError: 'dict' object has no attribute 'mean'

Support Vector Machines

In [96]:
clf = svm.SVC()
train_and_predict_using_pimodel('SVM (Classiefier)', clf)


SVM (Classiefier)
Training Confusion Matrix: 
[[52  0  0  7]
 [19  0  0  1]
 [14  0  0  0]
 [ 9  0  0 56]]
Training Accuracy: 68.35%
Testing Confusion Matrix: 
[[32  0  0  9]
 [11  0  0  1]
 [ 5  0  0  0]
 [ 6  0  0 34]]
Testing Accuracy: 67.35%


In [97]:
rbf_clf = svm.SVC(kernel='rbf')
train_and_predict_using_pimodel('SVM (Classiefier) - RBF', rbf_clf)

SVM (Classiefier) - RBF
Training Confusion Matrix: 
[[52  0  0  7]
 [19  0  0  1]
 [14  0  0  0]
 [ 9  0  0 56]]
Training Accuracy: 68.35%
Testing Confusion Matrix: 
[[32  0  0  9]
 [11  0  0  1]
 [ 5  0  0  0]
 [ 6  0  0 34]]
Testing Accuracy: 67.35%


In [98]:
rbf_svc = svm.SVC(kernel='poly')
train_and_predict_using_model('SVM (Classiefier) - Poly', rbf_svc)

SVM (Classiefier) - Poly
Training Confusion Matrix: 
[[48  0  0 32]
 [24  0  0  0]
 [ 2  0 14  0]
 [17  0  0 67]]
Training Accuracy: 63.24%
Testing Confusion Matrix: 
[[10  0  0 10]
 [ 8  0  0  0]
 [ 1  0  2  0]
 [ 2  0  0 19]]
Testing Accuracy: 59.62%


In [99]:
rbf_svc = svm.SVC(kernel='sigmoid')
train_and_predict_using_model('SVM (Classiefier) - Sigmoid', rbf_svc)

SVM (Classiefier) - Sigmoid
Training Confusion Matrix: 
[[42  0  0 38]
 [11  0  0 13]
 [ 0  0  0 16]
 [50  0  0 34]]
Training Accuracy: 37.25%
Testing Confusion Matrix: 
[[12  0  0  8]
 [ 4  0  0  4]
 [ 0  0  0  3]
 [ 9  0  0 12]]
Testing Accuracy: 46.15%
